In [1]:
from typing import Mapping, Iterator
import numpy as np
import os
from problog.logic import Term, Constant
from deepproblog.dataset import Dataset
from deepproblog.query import Query
import torch
import torchvision
import torchvision.transforms as transforms


In [121]:
class PuzzleDataset(Mapping[Term, torch.Tensor]):

    def __init__(self, subset: str, puzzle_path: str):
        """
        :param subset: Either 'train' or 'test'.
        :param puzzle_path: Path to the directory containing puzzle .npy files.
        :param label_path: Path to the directory containing label .npy files.
        """
        self.subset = subset
        self.puzzle_files = sorted(
            [f for f in os.listdir(puzzle_path) if f.endswith('.npy')]
        )
        self.puzzles = []
        for filename in self.puzzle_files:
            puzzle = np.load(os.path.join(puzzle_path, filename), allow_pickle=True)
            for i in range(len(puzzle)):
                for j in range(len(puzzle[i])):
                    if puzzle[i][j] is None:
                        puzzle[i][j] = np.zeros((28, 28), dtype=np.float32) / 255.0  # Create normalized zeros
                    else:
                        puzzle[i][j] = puzzle[i][j].astype(np.float32) / 255.0  # Normalize existing values
            self.puzzles.append(puzzle)

    def __len__(self) -> int:
        return len(self.puzzles)

    def __iter__(self) -> Iterator:
        for i in range(len(self)):
            yield self.puzzles[i]

    def __getitem__(self, item): #-> torch.Tensor:
        return torch.tensor(self.puzzles[int(item[0])][0][0]).unsqueeze(0)
        # return torch.from_numpy(self.puzzles[item])


In [111]:
class AdditionDataset(Dataset):

    def __init__(self, subset: str, labels_path: str):
        self.subset = subset
        self.label_files = sorted(
            [f for f in os.listdir(labels_path) if f.endswith('.npy')]
        )
        self.labels = []
        for filename in self.label_files:
            label = np.load(os.path.join(labels_path, filename), allow_pickle=True)
            for i in range(len(label)):
                for j in range(len(label[i])):
                    if label[i][j] is None:
                        label[i][j] = 0
            # label = label.astype(np.uint8)
            self.labels.append(label)

    def __len__(self):
        return len(self.labels) // 2

    def to_query(self, i: int) -> Query:
        image1_id = i*2
        image2_id = i*2+1
        image1_label = self.labels[image1_id].flatten()[0]
        image2_label = self.labels[image2_id].flatten()[0]
        image1 = Term("tensor", Term(self.subset, Constant(image1_id)))
        image2 = Term("tensor", Term(self.subset, Constant(image2_id)))
        label = Constant(int(image1_label) + int(image2_label))
        term = Term('addition', image1, image2, label)
        return Query(term)

ym = AdditionDataset('test', '../mnist_sudoku_generator/dataset/arrays/puzzles/test')
ym.to_query(0)

(1.0::addition(tensor(test(0)),tensor(test(1)),5), {})

In [86]:
import torch.nn as nn


class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.MaxPool2d(2, 2),  # 6 24 24 -> 6 12 12
            nn.ReLU(True),
            nn.Conv2d(6, 16, 5),  # 6 12 12 -> 16 8 8
            nn.MaxPool2d(2, 2),  # 16 8 8 -> 16 4 4
            nn.ReLU(True),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
            nn.Softmax(1),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1, 16 * 4 * 4)
        x = self.classifier(x)
        return x


In [118]:
data_path = '../mnist_sudoku_generator/dataset/images/puzzles/'
labels_path = '../mnist_sudoku_generator/dataset/arrays/puzzles/'

# train_dataset = PuzzleLabelDataset('train', data_path+'train', labels_path+'train')
test_dataset = PuzzleDataset('test', data_path+'test')
# print('train dataset size:', len(train_dataset))
print('test dataset size:', len(test_dataset))

test dataset size: 2000


In [119]:
test_dataset[[0]].shape

torch.Size([1, 28, 28])

In [ ]:
AdditionDataset('test', '../mnist_sudoku_generator/dataset/arrays/puzzles/test')

In [142]:
import torch

from deepproblog.dataset import DataLoader
from deepproblog.engines import ExactEngine
from deepproblog.model import Model
from deepproblog.network import Network
from deepproblog.train import train_model

network = MNIST_Net()
net = Network(network, "mnist_net", batching=True)
net.optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)

model = Model("addition.pl", [net])
model.set_engine(ExactEngine(model))
train = PuzzleDataset("train", "../mnist_sudoku_generator/dataset/images/puzzles/train")
test = PuzzleDataset("test", "../mnist_sudoku_generator/dataset/images/puzzles/test")
model.add_tensor_source("train", train)
model.add_tensor_source("test", test)

dataset = AdditionDataset('train', '../mnist_sudoku_generator/dataset/arrays/puzzles/train')


In [143]:
# Train the model
loader = DataLoader(dataset, 2, False)
train_model(model, loader, 1, log_iter=100, profile=0)
model.save_state("snapshot/trained_model.pth")

# Query the model
query = dataset.to_query(0)
result = model.solve([query])[0]
print(result)

Training  for 1 epoch(s)
Epoch 1
Iteration:  100 	s:5.2542 	Average Loss:  2.5576387731311843
Iteration:  200 	s:5.3462 	Average Loss:  1.7119511892792798
Iteration:  300 	s:5.4724 	Average Loss:  1.725659420330803
Iteration:  400 	s:4.9586 	Average Loss:  1.7328504049865114
Iteration:  500 	s:5.2718 	Average Loss:  1.6452261650564948
Iteration:  600 	s:5.0650 	Average Loss:  1.76128124415986
Iteration:  700 	s:5.3677 	Average Loss:  1.8094900826937186
Iteration:  800 	s:5.0815 	Average Loss:  1.6175137523793273
Iteration:  900 	s:5.3438 	Average Loss:  1.239225293014506
Iteration:  1000 	s:4.9532 	Average Loss:  1.017424865022666
Iteration:  1100 	s:5.3754 	Average Loss:  0.9075649040276267
Iteration:  1200 	s:5.3588 	Average Loss:  0.7864382118938056
Iteration:  1300 	s:5.0832 	Average Loss:  0.7062576016010348
Iteration:  1400 	s:5.3315 	Average Loss:  0.4756318800499355
Iteration:  1500 	s:5.0766 	Average Loss:  0.5739850180163603
Iteration:  1600 	s:5.4363 	Average Loss:  0.393991

In [147]:
from deepproblog.evaluate import get_confusion_matrix
dataset_test = AdditionDataset('test', '../mnist_sudoku_generator/dataset/arrays/puzzles/test')
cm = get_confusion_matrix(model, dataset_test)


In [156]:
cm.accuracy()

Accuracy:  0.933


np.float64(0.933)

In [160]:
print(str(cm))

         	  	  	  	  	   	  	  	  	Actual	  	  	  	  	  	  	  	  
         	  	 5	 6	 1	  0	 4	 2	 7	    11	10	13	 8	 9	 3	14	12	15
         	 5	94	 0	 0	  0	 0	 0	 0	     0	 2	 0	 1	 1	 2	 0	 0	 0
         	 6	 1	81	 0	  0	 4	 0	 1	     2	 0	 0	 0	 0	 0	 0	 0	 0
         	 1	 1	 1	53	  0	 0	 0	 3	     0	 0	 0	 2	 0	 0	 0	 0	 0
         	 0	 0	 0	 0	239	 0	 0	 0	     0	 0	 0	 0	 0	 0	 0	 0	 0
         	 4	 0	 0	 0	  0	63	 1	 1	     1	 0	 0	 0	 6	 0	 0	 0	 0
         	 2	 0	 1	 0	  0	 0	66	 2	     0	 0	 0	 1	 0	 2	 0	 0	 0
         	 7	 1	 0	 0	  0	 0	 1	65	     0	 0	 0	 2	 2	 0	 0	 0	 0
Predicted	11	 0	 0	 0	  0	 0	 0	 0	    23	 0	 1	 0	 0	 0	 0	 0	 0
         	10	 0	 0	 0	  0	 0	 0	 0	     2	38	 0	 0	 0	 0	 0	 1	 0
         	13	 0	 0	 0	  0	 0	 0	 0	     2	 0	11	 0	 0	 0	 0	 0	 0
         	 8	 0	 0	 0	  0	 0	 1	 0	     0	 0	 2	54	 2	 0	 2	 0	 1
         	 9	 0	 0	 0	  0	 1	 0	 0	     0	 2	 0	 0	30	 1	 0	 0	 0
         	 3	 2	 0	 0	  0	 0	 0	 1	     0	 1	 0	 1	 0	84	 0	 0	 0
         	

In [144]:
from PIL import Image
Image.fromarray(test[[2]].numpy().reshape(28, 28) * 255).show()
Image.fromarray(test[[3]].numpy().reshape(28, 28) * 255).show()

In [108]:
from typing import Mapping, Iterator

import torch
import torchvision
import torchvision.transforms as transforms
from problog.logic import Term, Constant

from deepproblog.dataset import Dataset
from deepproblog.query import Query

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

datasets = {
    "train": torchvision.datasets.MNIST(
        root='data/', train=True, download=True, transform=transform
    ),
    "test": torchvision.datasets.MNIST(
        root='data/', train=False, download=True, transform=transform
    ),
}


class MNISTImages(Mapping[Term, torch.Tensor]):

    def __iter__(self) -> Iterator:
        for i in range(self.dataset):
            yield self.dataset[i][0]

    def __len__(self) -> int:
        return len(self.dataset)

    def __init__(self, subset):
        self.subset = subset
        self.dataset = datasets[self.subset]

    def __getitem__(self, item):
        print(self.dataset[int(item[0])][0])
        return self.dataset[int(item[0])][0]


class AdditionDataset(Dataset):

    def __init__(self, subset):
        self.subset = subset
        self.dataset = datasets[subset]

    def __len__(self):
        return len(self.dataset) // 2

    def to_query(self, i: int) -> Query:
        image1 = Term("tensor", Term(self.subset, Constant(i * 2)))
        image2 = Term("tensor", Term(self.subset, Constant(i * 2 + 1)))
        label = Constant(int(self.dataset[i*2][1] + self.dataset[i*2+1][1]))
        term = Term('addition', image1, image2, label)
        return Query(term)

ym = MNISTImages('train')
ym[[0]].shape

tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1

torch.Size([1, 28, 28])